# Aumento de datos utilizando redes generativas para mejorar el reconocimiento y segmentación de imágenes de microscopía óptica.

En este notebook yadda yadda

## Sección 0: StyleGAN 2
Lo primero que se necesita para poner a funcionar este sistema es un checkpoint de la versión vanilla de StyleGAN 2 ([esta](https://github.com/dvschultz/stylegan2)), previamente entrenado para producir imágenes fotorealistas de nematodos. Es necesario convertir el checkpoint a formato pytorch; esto se puede hacer con un script que se puede encontrar en la versión de pytorch de StyleGAN 2 ([esta](https://github.com/rosinality/stylegan2-pytorch)). Un ejemplo de como hacer esta conversión se puede encontrar [aquí](https://github.com/dvschultz/ai/blob/master/Convert_pkl_to_pt.ipynb).

Cabe también mencionar que la versión vanilla de StyleGAN usa una versión obsoleta de tensorflow. Afortunadamente, nvidia todavía mantiene compatibilidad por medio de un contenedor de docker (ver https://catalog.ngc.nvidia.com/orgs/nvidia/containers/tensorflow), lo que permite hacer uso de este sistema.

Para simplificar la implementación, se asumirá que el checkpoint estará en la dirección .checkpoint/stylegan2/network-snapshot.pt, si se desea usar otra dirección u otro nombre para el checkpoint se deben modificar los files de configuración tool_nema.json, encoder_nema.json y datasetgan_nema.json en el directorio experiments.

## Sección 1: EditGAN

### Entrenar el encoder

El primer paso es entrenar el encoder. Se asume que las imágenes de entrenamiento tendrán un tamaño de 256x256 y estarán en data/encoder; si se desea usar imágenes con otro tamaño se puede modificar el parámetro im_size en experiments/encoder_nema.json pero es importante notar que la resolución de la imágen tiene que ser consistente para todo el sistema y StyleGAN 2 solo es capaz de generar imágenes cuadradas donde el tamaño de su lado es una potencia de dos.

Una importante ventaja de este entrenamiento es que no es necesario utilizar exclusivamente imágenes reales: el propósito del encoder es reproducir imágenes de entrada en el espacio latente de StyleGAN 2 así que para entrenarlo es posible utilizar un conjunto de imágenes sintéticas arbitrariamente grande, producido por el mismo StyleGAN 2.

In [ ]:
!python ../train_encoder.py --exp experiments/encoder_nema.json

El comando para continuar con el entrenamiento del encoder a partir de un checkpoint es:

<code>  train_encoder.py --exp experiments/encoder_nema.json --resume checkpoint_path.pth  </code>

### Preparar un conjunto para datasetGAN

El siguiente paso es preparar un pequeño conjunto de parejas imagen-máscara para entrenar a datasetGAN. Este entrenamiento suele requerir de mucha RAM así que se recomiendo utilizar 16 parejas o menos.

Las imágenes deben ser nombradas según el formato image_#.png y las máscaras deben ser convertidas a numpy array y deben ser nombradas según el formato image_mask_#.npy; ambos grupos deben ser guardados en el mismo directorio. La siguiente celda se ocupará de tomar un conjunto aleatorio de pares imagen-máscara, de los etiquetados manualmente, y las guardará en el formato correcto.

Por default los pares se guardarán en la carpeta extra_utils/data/datasetgan_pairs. Si se deseara cambiar esta dirección sería necesario modificar el config experiments/datasetgan_nema.json

In [ ]:
from data_processor import process_data

# Dirección de la carpeta donde se encuentran las imágenes de los nematodos.
img_in = '/home/marco/Desktop/proyecto_graduacion/etiquetados/simple_experiment/img_edits'

# Dirección de la carpeta donde se encuentran las máscaras.
mask_in = '/home/marco/Desktop/proyecto_graduacion/etiquetados/simple_experiment/mask_edits'

# Dirección donde guardar los pares imagen-máscara procesados
img_out = 'data/datasetgan_pairs'

# Cantidad de pares imagen-máscara que muestrear
sample_size = 16

process_data(img_in, mask_in, img_out, sample_size)

### Introducir conjunto en el espacio latente

Lo que sigue es introducir el conjunto que se acaba de crear en el espacio latente de StyleGAN 2. Para esto es necesario que en el siguiente código se reemplace BEST_loss.pth con el nombre del checkpoint del encoder que se va a utilizar. Es necesario hacer este mismo cambio también en el config experiments/tool_nema.json.

In [ ]:
!python ../python train_encoder.py --exp experiments/encoder_nema.json --resume ../model_encoder/nema/checkpoint/BEST_loss.pth --testing_path data/datasetgan_pairs --latent_sv_folder ../model_encoder/nema/training_embedding --test True 

### Entrenar a DatasetGAN

Este es el último entrenamiento de EditGAN, y es el que requiere de mucha RAM.

In [ ]:
!python ../train_interpreter.py --exp experiments/datasetgan_nema.json

<div class="alert alert-block alert-info">
<b>Tip:</b> Si el entrenamiento requiere de más RAM de la que se tiene a disposición, igual es posible realizarlo utilizando memoria virtual. Esta es una solución imperfecta y resultará en un entrenamiento tremendamente lento, pero fue lo que permitió realizarlo con 16 pares. Se requiere de un espacio swap suficientemente grande y se realiza con el comando:
    
<code>  systemd-run --scope -p MemoryMax=26G python train_interpreter.py --exp experiments/datasetgan_nema.json  </code>
    
Donde MemoryMax es la máxima cantidad de RAM que se le permite usar antes de usar swap.
    
</div>




### Generar el conjunto sintético anotado

Finalmente, EditGAN está listo para generar el conjunto etiquetado. Se recomienda a este punto generar un conjunto muy grande porque por el momento es de esperarse que salgan muchas máscaras de mala calidad.

In [ ]:
import sys
sys.path.append('../')


import dataset_gen 

sampling_amount = 128

img_out_path = 'data/test_img'

mask_out_path = 'data/test_mask'

dataset_gen.mode_1(sampling_amount, img_out_path, mask_out_path)

Podemos darnos una idea de la calidad del conjunto generado creando un mosácio:

In [ ]:
from mosaic_maker import create_mosaic

img_in = '/home/marco/Desktop/proyecto_graduacion/editGAN_release/dataset_final/simple/img'

mask_in = '/home/marco/Desktop/proyecto_graduacion/editGAN_release/dataset_final/simple/mask'

img_num = 64

create_mosaic(img_in, mask_in, img_num, img_out)

Es muy probable que a este punto la calidad de la mayoría de las máscaras deje bastante que desear. A continuación se aplicará un filtro a este conjunto recién creado para elegir solamente las mejores máscaras.

## Sección 2: el filtro

El filtro consiste de dos partes:

1. Un primer filtrado hecho a partir del tamaño del contorno vermíforme. En el mosáico de arriba debería resultar evidente que las máscaras que se pueden descartar inmediatamente tienden a ser una colección de pequeñas manchas. Un umbral de tamaño mínimo se ocupa de eliminar todas esas y de elegir solamente la forma del nematodo en las que están bien pero visualmente ruidosas.
2. Un support vector machine  que intenta determinar cuales máscaras son buenas y cuales malas a partir de los 7 momentos de Hu del contorno del nematodo, de su perímetro y de la magnitud de las 5 componentes de mayor frecuencia de la transformada de fourier de la máscara.

Es inevitable a este punto que para entrenar al SVM sea necesario elegir manualmente a un conjunto de máscaras malas y un conjunto de máscaras buenas.

El entrenamiento del SVM ser hará con un grid search así que es necesario definir sus parámetros. También digno de nota es que para entrenar el SVM se está usando MinMaxScaler, y ese mismo scaler debe ser utilizado a la hora de hacer inferencia así que se guardará junto con el.

NOTA: el grid search está configurado para usar todos los recursos de la computadora que pueda. Si esto no es deseable cambiar la variable n_jobs en train_svm.

In [ ]:
from svm_trainer import train_svm

# Dirección de las máscaras de buena calidad
good_path = '' 

# Dirección de las máscaras de mala calidad
bad_path = ''

# Dirección donde guardar el SVM y el scaler
save_path = '../.checkpoint/SVM'

param_grid = {'C':[1,4,5,6,10],'gamma':[50,85, 90, 100, 110, 115],'kernel':['sigmoid', 'rbf','poly']}

Con el SVM entrenado se aplica el filtro:

In [ ]:
from image_filter import process_image

# Este código es un poco obtuso pero fue hecho de 
# esta manera para paralelizar fácilmente

# Dirección donde se encuentran las imágenes por filtrar.
img_in = ''

# Dirección donde se encuentran las máscaras por filtrar.
mask_in = ''

# Dirección donde guardar las imágenes que son aceptadas por el filtro. 
img_out = ''

# Dirección donde guardar las máscaras que son aceptadas por el filtro.
mask_out = ''

# Dirección donde guardar las máscaras que son rechazadas por el filtro. 
# Si es None, no guarda estas máscaras.
bad_out = ''

# Dirección del SVM
svm_path = '../.checkpoint/SVM'

# Dirección del scaler
scaler_path = '../.checkpoint/SVM'

# Umbral de probabilidad para que el SVM acepte una máscara.
svm_proba = 0.8

# Umbral de área ocupada por el namatodo. Máscaras con un area inferior serán rechazadas.
min_area = 2600

masknames = next(walk(mask_in), (None, None, []))[2]  # [] if no file
masknames.sort()

model = pickle.load(open(svm_path, 'rb'))
scaler = pickle.load(open(scaler_path, 'rb'))

param = { 
    'img_in': img_in,
    'mask_in': mask_in,
    'img_out': img_out,
    'mask_out': mask_out,
    'bad_out': bad_out,
    'min_area': min_area,
    'model': model,
    'scaler': scaler,
    'svm_proba': svm_proba,
    'mask_name': ''
    }

params = []

for i in masknames:
    temp = param.copy()
    temp['mask_name'] = i
    params.append(temp)


t1 = time.perf_counter()

with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(process_image, params) 


t2 = time.perf_counter()
time_dif = (t2-t1)/(60*60)

print(f'Finished in {time_dif} hours')

Habiendo filtrado los datos, podemos volver a generar un mosáico para ver el conjunto final:

In [ ]:
from mosaic_maker import create_mosaic

img_num = 64

create_mosaic(img_in, mask_in, img_num, img_out)